In [1]:
import os, glob, math, warnings
import numpy as np
import h5py
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from tqdm import tqdm

In [ ]:
import os, pickle, warnings
import numpy as np
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from tqdm import tqdm

# -----------------------------
# Config
# -----------------------------
DATA_DIR = "data"  # folder containing s01.dat ... s32.dat
SUBJECTS = [f"s{idx:02d}.dat" for idx in range(1, 33)]

FS = 128
TRIAL_SEC = 60
BASELINE_SEC = 3
KEEP_SAMPLES = FS * TRIAL_SEC              # 7680
DROP_SAMPLES = FS * BASELINE_SEC           # 384 (drop from start)

EEG_CH = 32                                # first 32 channels are EEG
BANDS = [(4,8), (8,13), (13,30), (30,45)]  # theta, alpha, beta, gamma
LABELS = ["valence", "arousal", "dominance", "liking"]
TARGET = "valence"                         # change to "arousal"/"liking"/"dominance" as needed

# We'll use 4 robust peripheral channels commonly present after EEG:
#  GSR, Respiration, BVP, Skin Temp (indices relative to *peripheral* block)
PERIPH_KEEP = [0, 1, 2, 3]

# -----------------------------
# Helpers
# -----------------------------
def binarize(y_cont):
    """DEAP convention: rating > 5 => 1 (High), else 0 (Low)."""
    return (y_cont > 5.0).astype(np.int32)

def load_subject_dat(path):
    """
    Load DEAP preprocessed .dat (pickle).
    Returns:
      eeg:    (40, 32, 7680)
      periph: (40, P, 7680) or None
      labels: (40, 4)
    """
    with open(path, "rb") as f:
        obj = pickle.load(f, encoding="latin1")  # DEAP pickles were py2
    data = obj["data"]         # shape (40, 40, 8064): trials, channels, samples
    labels = obj["labels"]     # shape (40, 4): valence, arousal, dominance, liking (1..9)

    # split channels
    eeg = data[:, :EEG_CH, :]                  # (40, 32, 8064)
    periph = data[:, EEG_CH:, :]               # (40, 8, 8064)

    # keep only the 60s trial segment (drop first 3s baseline)
    eeg = eeg[:, :, DROP_SAMPLES:DROP_SAMPLES + KEEP_SAMPLES]          # (40, 32, 7680)
    periph = periph[:, :, DROP_SAMPLES:DROP_SAMPLES + KEEP_SAMPLES]    # (40, 8, 7680)

    # keep robust peripheral subset
    if periph.shape[1] >= 4:
        periph = periph[:, PERIPH_KEEP, :]
    else:
        periph = None

    return eeg.astype(np.float32), (periph.astype(np.float32) if periph is not None else None), labels.astype(np.float32)

def welch_logbp(x, fs=FS, nperseg=256, noverlap=128):
    f, Pxx = welch(x, fs=fs, nperseg=nperseg, noverlap=noverlap)
    out = []
    for lo, hi in BANDS:
        m = (f >= lo) & (f <= hi)
        bp = np.trapz(Pxx[m], f[m]) + 1e-12
        out.append(np.log(bp))
    return np.asarray(out, dtype=np.float32)

def eeg_features(trial_eeg):
    # trial_eeg: (32, 7680)
    feats = [welch_logbp(trial_eeg[ch]) for ch in range(trial_eeg.shape[0])]
    return np.concatenate(feats, axis=0)  # (32 * 4,)

def periph_features(trial_periph):
    # trial_periph: (P, 7680); simple trial-level stats
    def slope(x):
        t = np.arange(x.size, dtype=np.float32)
        A = np.vstack([t, np.ones_like(t)]).T
        m, _ = np.linalg.lstsq(A, x, rcond=None)[0]
        return np.float32(m)
    feats = []
    for ch in range(trial_periph.shape[0]):
        sig = trial_periph[ch]
        feats.extend([np.mean(sig), np.std(sig), slope(sig)])
    return np.asarray(feats, dtype=np.float32)  # (P*3,)

def build_features(eeg, periph):
    X_eeg = np.vstack([eeg_features(eeg[i]) for i in range(eeg.shape[0])])
    X_per = None
    if periph is not None:
        X_per = np.vstack([periph_features(periph[i]) for i in range(periph.shape[0])])
    return X_eeg, X_per

def metric_dict(y_true, y_pred):
    return {
        "acc": float(accuracy_score(y_true, y_pred)),
        "bacc": float(balanced_accuracy_score(y_true, y_pred)),
        "f1_macro": float(f1_score(y_true, y_pred, average="macro"))
    }

def subject_dependent_cv(X, y, C=1.0):
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    scores = []
    for tr, te in skf.split(X, y):
        scaler = StandardScaler()
        Xtr = scaler.fit_transform(X[tr])
        Xte = scaler.transform(X[te])
        base = LinearSVC(C=C)
        clf = CalibratedClassifierCV(base, method="sigmoid", cv=3)
        clf.fit(Xtr, y[tr])
        yhat = clf.predict(Xte)
        scores.append(metric_dict(y[te], yhat))
    return {k: float(np.mean([s[k] for s in scores])) for k in scores[0]}

def loso(mods, labels, C=1.0):
    """
    mods: dict modality -> list of per-subject X arrays
    labels: list of per-subject y arrays
    returns averaged metrics per modality and for fusion
    """
    subj_n = len(labels)
    res = {k: [] for k in mods.keys()}
    res["fusion"] = []

    for te in range(subj_n):
        # set up per-modality models on train subjects
        permod = {}
        for mod, Xs in mods.items():
            Xtr = np.vstack([Xs[i] for i in range(subj_n) if i != te])
            ytr = np.hstack([labels[i] for i in range(subj_n) if i != te])
            Xte = Xs[te]
            yte = labels[te]

            scaler = StandardScaler()
            Xtr = scaler.fit_transform(Xtr)
            Xte = scaler.transform(Xte)

            base = LinearSVC(C=C)
            clf = CalibratedClassifierCV(base, method="sigmoid", cv=3)
            clf.fit(Xtr, ytr)
            yhat = clf.predict(Xte)
            proba = clf.predict_proba(Xte)
            permod[mod] = (yte, yhat, proba)

        # per-modality metrics
        for mod, (yte, yhat, proba) in permod.items():
            res[mod].append(metric_dict(yte, yhat))

        # simple late fusion: average calibrated probabilities
        probs = np.mean(np.stack([v[2] for v in permod.values()], axis=0), axis=0)
        yhat_f = np.argmax(probs, axis=1)
        yte0 = next(iter(permod.values()))[0]
        res["fusion"].append(metric_dict(yte0, yhat_f))

    return {k: {m: float(np.mean([r[m] for r in v])) for m in v[0]} for k, v in res.items()}

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    warnings.filterwarnings("ignore")

    all_eeg, all_per, all_y = [], [], []

    print("Loading .dat files and extracting features …")
    for fname in tqdm(SUBJECTS):
        fpath = os.path.join(DATA_DIR, fname)
        if not os.path.exists(fpath):
            continue
        eeg, periph, labels = load_subject_dat(fpath)
        Xeeg, Xper = build_features(eeg, periph)

        # choose target label and binarize
        t_idx = LABELS.index(TARGET)
        y = binarize(labels[:, t_idx])

        all_eeg.append(Xeeg)
        if Xper is not None:
            all_per.append(Xper)
        else:
            all_per.append(np.zeros((Xeeg.shape[0], 0), dtype=np.float32))
        all_y.append(y)

    # ---- Subject-dependent baseline (EEG only) ----
    sub_metrics = []
    for i in range(len(all_eeg)):
        sub_metrics.append(subject_dependent_cv(all_eeg[i], all_y[i], C=1.0))
    avg_sub = {k: float(np.mean([m[k] for m in sub_metrics])) for k in sub_metrics[0]}
    print("\n=== Subject-dependent (EEG) — 10-fold within subject ===")
    print(avg_sub)

    # ---- LOSO across subjects (EEG, peripherals, fusion) ----
    mods = {"eeg": all_eeg, "periph": all_per}
    loso_avg = loso(mods, all_y, C=1.0)
    print("\n=== LOSO Cross-Subject ===")
    for k, v in loso_avg.items():
        print(k, v)


In [10]:

# -----------------------------
# Config
# -----------------------------
DATA_DIR = "datasets/DEAP/deap-dataset/data_preprocessed_python"
FS = 128                           # preprocessed sampling rate
EEG_CH_COUNT = 32
TRIAL_LEN_SEC = 60
TRIAL_SAMPLES = FS * TRIAL_LEN_SEC # 7680
BANDS = [(4,8), (8,13), (13,30), (30,45)]  # theta, alpha, beta, gamma
LABEL_NAMES = ["valence", "arousal", "dominance", "liking"]
TARGET = "valence"                 # change to "arousal" or "liking" as needed
SUBJECTS = [f"s{idx:02d}.dat" for idx in range(1,33)]

# Peripheral channels (DEAP preprocessed order reference)
# EEG: 32 channels first; then 8 peripherals in common mirrors:
#  32: GSR, 33: Respiration belt, 34: Plethysmograph (BVP),
#  35: Temp, 36-39: EOG/EMG variants depending on mirror (often 4 channels).
# We’ll pick the core 4 peripherals that are consistent across mirrors.
PERIPH_IDXS = None  # will set after we parse a file

In [11]:


# -----------------------------
# Utilities
# -----------------------------
def binarize_ratings(y_cont):
    # DEAP convention used widely: >5 => High; else Low
    return (y_cont > 5.0).astype(int)

def load_subject_mat(path):
    """
    Loads a DEAP preprocessed subject .mat (HDF5 or MATLAB v7.3).
    Returns:
      eeg: shape (40, 32, 8064)
      periph: shape (40, P, 8064) with P peripheral channels if available else None
      labels: shape (40, 4)
    """
    with h5py.File(path, 'r') as f:
        # DEAP files vary between mirrors; try common layouts
        if 'data' in f:
            d = np.array(f['data'])
            # Many mirrors store as (40 trials, 40 channels, 8064 samples)
            # or (40, 32, 8064) + peripherals after 32.
            if d.ndim == 3:
                trials, chans, samples = d.shape
                eeg = d[:, :EEG_CH_COUNT, :]
                periph = d[:, EEG_CH_COUNT:, :] if chans > EEG_CH_COUNT else None
            elif d.ndim == 2:
                # Rare: flattened; not common — skip
                raise RuntimeError("Unexpected 2D data array")
            else:
                raise RuntimeError("Unknown data layout in .mat")
        else:
            raise RuntimeError("No 'data' key found in .mat file")

        if 'labels' in f:
            labels = np.array(f['labels'])
        elif 'label' in f:
            labels = np.array(f['label'])
        else:
            raise RuntimeError("No labels found in .mat")

        # labels may be (4,40) — transpose to (40,4)
        if labels.shape[0] == 4 and labels.shape[1] == 40:
            labels = labels.T
        return eeg.astype(np.float32), (periph.astype(np.float32) if periph is not None else None), labels.astype(np.float32)

def welch_logpsd(x, fs=128, nperseg=256, noverlap=128):
    """
    x: (samples,) 1D signal
    returns log bandpowers for BANDS
    """
    f, Pxx = welch(x, fs=fs, nperseg=nperseg, noverlap=noverlap)
    feat = []
    for (lo, hi) in BANDS:
        mask = (f >= lo) & (f <= hi)
        bp = np.trapz(Pxx[mask], f[mask]) + 1e-12
        feat.append(np.log(bp))
    return np.array(feat, dtype=np.float32)

def extract_eeg_features(trial_eeg):
    """
    trial_eeg: (32, samples)
    return: (32 * len(BANDS),) bandpower features
    """
    feats = [welch_logpsd(trial_eeg[ch]) for ch in range(trial_eeg.shape[0])]
    return np.concatenate(feats, axis=0)

def extract_periph_features(trial_periph):
    """
    trial_periph: (P, samples)
    Simple stats per channel (mean, std, slope, peakrate for BVP/GSR-ish)
    """
    def slope(x):
        t = np.arange(len(x))
        # least squares slope
        A = np.vstack([t, np.ones_like(t)]).T
        m, _ = np.linalg.lstsq(A, x, rcond=None)[0]
        return m

    feats = []
    for ch in range(trial_periph.shape[0]):
        sig = trial_periph[ch]
        feats.extend([np.mean(sig), np.std(sig), slope(sig)])
    return np.array(feats, dtype=np.float32)

def build_modality_features(eeg, periph):
    X_eeg = np.vstack([extract_eeg_features(eeg[i]) for i in range(eeg.shape[0])])
    X_per = None
    if periph is not None and periph.shape[1] > 0:
        X_per = np.vstack([extract_periph_features(periph[i]) for i in range(periph.shape[0])])
    return X_eeg, X_per

def metrics(y_true, y_pred):
    return {
        "acc": accuracy_score(y_true, y_pred),
        "bacc": balanced_accuracy_score(y_true, y_pred),
        "f1_macro": f1_score(y_true, y_pred, average="macro")
    }

def train_eval_subject_dependent(X, y):
    """
    10-fold CV within subject (40 trials) -> average metrics
    """
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    mm = []
    for tr, te in skf.split(X, y):
        scaler = StandardScaler()
        Xtr = scaler.fit_transform(X[tr])
        Xte = scaler.transform(X[te])
        base = LinearSVC(C=1.0)
        clf = CalibratedClassifierCV(base, method="sigmoid", cv=3)
        clf.fit(Xtr, y[tr])
        yhat = clf.predict(Xte)
        mm.append(metrics(y[te], yhat))
    # average
    out = {k: float(np.mean([m[k] for m in mm])) for k in mm[0]}
    return out

def train_eval_LOSO(mods, labels):
    """
    Leave-one-subject-out across 32 subjects. mods is a dict of modality→list_per_subject_of_X
    labels: list_per_subject_of_y
    - Trains calibrated LinearSVC per modality on 31 subjects
    - Decision-level fusion: average predicted probabilities
    """
    subjects = len(labels)
    results = {k: [] for k in mods.keys()}
    results["fusion"] = []

    for test_idx in range(subjects):
        # Build train/test for each modality
        per_mod_train = {}
        per_mod_test  = {}
        for mod, X_list in mods.items():
            Xtr = np.vstack([X_list[i] for i in range(subjects) if i != test_idx])
            Xte = X_list[test_idx]
            ytr = np.hstack([labels[i] for i in range(subjects) if i != test_idx])
            yte = labels[test_idx]

            scaler = StandardScaler()
            Xtr = scaler.fit_transform(Xtr)
            Xte = scaler.transform(Xte)

            base = LinearSVC(C=1.0)
            clf = CalibratedClassifierCV(base, method="sigmoid", cv=3)
            clf.fit(Xtr, ytr)
            yhat = clf.predict(Xte)
            proba = clf.predict_proba(Xte)  # (N,2)

            per_mod_train[mod] = (clf, ytr)  # keep if needed
            per_mod_test[mod]  = (yte, yhat, proba)

        # per-modality metrics
        for mod, (yte, yhat, proba) in per_mod_test.items():
            results[mod].append(metrics(yte, yhat))

        # fusion: average probs over available modalities
        probas = [v[2] for v in per_mod_test.values()]
        P = np.mean(np.stack(probas, axis=0), axis=0)
        yhat_fus = np.argmax(P, axis=1)
        yte0 = list(per_mod_test.values())[0][0]
        results["fusion"].append(metrics(yte0, yhat_fus))

    # average metrics
    avg = {k: {m: float(np.mean([r[m] for r in v])) for m in v[0]} for k, v in results.items()}
    return avg

# -----------------------------
# Main: build subject feature sets and run experiments
# -----------------------------
warnings.filterwarnings("ignore")
all_eeg, all_per, all_y = [], [], []

print("Loading subjects & extracting features …")
for sname in tqdm(SUBJECTS):
    fpath = os.path.join(DATA_DIR, sname)
    if not os.path.exists(fpath):
        continue
    eeg, periph, labels = load_subject_mat(fpath)

    # normalize shapes: many mirrors store (40, channels, samples)
    if eeg.shape[-1] != TRIAL_SAMPLES and eeg.shape[-1] > TRIAL_SAMPLES:
        # Some mirrors include 3 s baseline (384 samples) before the 60 s trial.
        # Keep last 60 s.
        eeg = eeg[..., -TRIAL_SAMPLES:]
        if periph is not None:
            periph = periph[..., -TRIAL_SAMPLES:]

    # choose peripheral indices if present
    if periph is not None and PERIPH_IDXS is None:
        # Try to keep first 4 peripheral channels (GSR, Resp, BVP, Temp)—robust across mirrors
        P = periph.shape[1]
        PERIPH_IDXS = list(range(min(P, 4)))

    # build features per trial
    Xeeg, Xper = build_modality_features(eeg, (periph[:, PERIPH_IDXS, :] if (periph is not None and len(PERIPH_IDXS)>0) else None))
    all_eeg.append(Xeeg)
    all_per.append(Xper if Xper is not None else np.zeros((eeg.shape[0], 0), dtype=np.float32))

    # labels → pick target and binarize
    idx = LABEL_NAMES.index(TARGET)
    y = binarize_ratings(labels[:, idx])
    all_y.append(y)

# Subject-dependent (per subject) EEG baseline (classic)
subdep_metrics = []
for i in range(len(all_eeg)):
    mm = train_eval_subject_dependent(all_eeg[i], all_y[i])
    subdep_metrics.append(mm)
avg_subdep = {k: float(np.mean([m[k] for m in subdep_metrics])) for k in subdep_metrics[0]}

# LOSO with EEG, peripherals, and fusion
mods = {"eeg": all_eeg, "periph": all_per}
loso_avg = train_eval_LOSO(mods, all_y)

print("\n=== Subject-dependent (EEG) — 10-fold within subject ===")
print(avg_subdep)

print("\n=== LOSO Cross-Subject ===")
for k, v in loso_avg.items():
    print(k, v)


In [12]:

# -----------------------------
# Main: build subject feature sets and run experiments
# -----------------------------
warnings.filterwarnings("ignore")
all_eeg, all_per, all_y = [], [], []

print("Loading subjects & extracting features …")
for sname in tqdm(SUBJECTS):
    fpath = os.path.join(DATA_DIR, sname)
    print(fpath)
    if not os.path.exists(fpath):
        continue
    eeg, periph, labels = load_subject_mat(fpath)

    # normalize shapes: many mirrors store (40, channels, samples)
    if eeg.shape[-1] != TRIAL_SAMPLES and eeg.shape[-1] > TRIAL_SAMPLES:
        # Some mirrors include 3 s baseline (384 samples) before the 60 s trial.
        # Keep last 60 s.
        eeg = eeg[..., -TRIAL_SAMPLES:]
        if periph is not None:
            periph = periph[..., -TRIAL_SAMPLES:]

    # choose peripheral indices if present
    if periph is not None and PERIPH_IDXS is None:
        # Try to keep first 4 peripheral channels (GSR, Resp, BVP, Temp)—robust across mirrors
        P = periph.shape[1]
        PERIPH_IDXS = list(range(min(P, 4)))

    # build features per trial
    Xeeg, Xper = build_modality_features(eeg, (periph[:, PERIPH_IDXS, :] if (periph is not None and len(PERIPH_IDXS)>0) else None))
    all_eeg.append(Xeeg)
    all_per.append(Xper if Xper is not None else np.zeros((eeg.shape[0], 0), dtype=np.float32))

    # labels → pick target and binarize
    idx = LABEL_NAMES.index(TARGET)
    y = binarize_ratings(labels[:, idx])
    all_y.append(y)

# Subject-dependent (per subject) EEG baseline (classic)
subdep_metrics = []
for i in range(len(all_eeg)):
    mm = train_eval_subject_dependent(all_eeg[i], all_y[i])
    subdep_metrics.append(mm)
avg_subdep = {k: float(np.mean([m[k] for m in subdep_metrics])) for k in subdep_metrics[0]}

# LOSO with EEG, peripherals, and fusion
mods = {"eeg": all_eeg, "periph": all_per}
loso_avg = train_eval_LOSO(mods, all_y)

print("\n=== Subject-dependent (EEG) — 10-fold within subject ===")
print(avg_subdep)

print("\n=== LOSO Cross-Subject ===")
for k, v in loso_avg.items():
    print(k, v)


Loading subjects & extracting features …


  0%|          | 0/32 [00:00<?, ?it/s]

datasets/DEAP/deap-dataset/data_preprocessed_python/s01.dat


OSError: Unable to synchronously open file (file signature not found)